## 1. 데이터 확인

### 필수 라이브러리

In [102]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore') # 경고 메세지 무시

In [103]:
df_train_origin = pd.read_csv("./data/train.csv") # 학습용 데이터
df_test_origin = pd.read_csv("./data/submission.csv") # 테스트 데이터(제출파일의 데이터)

In [104]:
df_train_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         58317 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            15338 non-null  object 
 6   enterprise               59299 non-null  object 
 7   historical_existing_cnt  13756 non-null  float64
 8   id_strategic_ver         3444 non-null   float64
 9   it_strategic_ver         1121 non-null   float64
 10  idit_strategic_ver       4565 non-null   float64
 11  customer_job             40566 non-null  object 
 12  lead_desc_length         59299 non-null  int64  
 13  inquiry_type             58358 non-null  object 
 14  product_category      

## 2. 데이터 전처리

### 각 변수별 확인

In [105]:
df_train_process = pd.read_csv('./data/Ch2/df_train.csv')
df_test_process = pd.read_csv('./data/Ch2/df_test.csv')

In [106]:
df_train_process.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         59299 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            59299 non-null  object 
 6   enterprise               59299 non-null  object 
 7   historical_existing_cnt  59299 non-null  float64
 8   id_strategic_ver         59299 non-null  float64
 9   it_strategic_ver         59299 non-null  float64
 10  idit_strategic_ver       59299 non-null  float64
 11  customer_job             59299 non-null  object 
 12  lead_desc_length         59299 non-null  int64  
 13  inquiry_type             59299 non-null  object 
 14  product_category      

## 3. 피처엔지니어링

### 레이블 인코딩

In [107]:
df_train_encoded = pd.read_csv('./data/Ch3/df_train_encoded.csv')
df_test_encoded = pd.read_csv('./data/Ch3/df_test_encoded.csv')

In [108]:
df_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   bant_submit               59299 non-null  float64
 1   customer_country          59299 non-null  int64  
 2   business_unit             59299 non-null  int64  
 3   com_reg_ver_win_rate      59299 non-null  float64
 4   customer_idx              59299 non-null  int64  
 5   customer_type             59299 non-null  int64  
 6   enterprise                59299 non-null  int64  
 7   historical_existing_cnt   59299 non-null  float64
 8   customer_job              59299 non-null  int64  
 9   lead_desc_length          59299 non-null  int64  
 10  inquiry_type              59299 non-null  int64  
 11  customer_country.1        59299 non-null  int64  
 12  customer_position         59299 non-null  int64  
 13  response_corporate        59299 non-null  int64  
 14  expect

## 4. 모델 학습

### 데이터 분할

학습, 검증 데이터 분리

In [109]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(
    df_train_encoded.drop("is_converted", axis=1),
    df_train_encoded["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

### 모델 라이브러리

단일모델 기준으로 사용할수 있는 모델들의 라이브러리를 불러오는 코드에 작성

In [110]:
# 모델 라이브러리
## 단일모델로는 이와같은 모델정도 있으니깐 참고
from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier

# 보팅
from sklearn.ensemble import VotingClassifier

# 스테킹
from sklearn.ensemble import StackingClassifier

### 최적 하이퍼 파라미터 찾기

optuna를 통한 최적의 파라미터 찾기

In [111]:
## 파라미터 찾는 과정.. ##

다시한번 optuna를 통한 최적의 파라미터 찾는과정

### 모델 정의

In [112]:
### 단일 모델 ###

# LightGBM
model = LGBMClassifier(
    num_leaves=106
    , max_depth=19
    , n_estimators=409
    , learning_rate=0.06987263978834855
    , min_child_samples=27
    , verbose = -1
)


### 모델 학습

In [113]:
model.fit(x_train, y_train)

LGBMClassifier(learning_rate=0.06987263978834855, max_depth=19,
               min_child_samples=27, n_estimators=409, num_leaves=106,
               verbose=-1)

### 모델 성능 보기

In [114]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])
    micro_F1 = f1_score(y_test, y_pred, average='micro')  # 추가된 부분

    metrics = pd.DataFrame({
        '정확도': [accuracy],
        '정밀도': [precision],
        '재현율': [recall],
        'F1 Score': [F1],
        'Micro F1': [micro_F1]  # 추가된 부분
    })

    confusion_df = pd.DataFrame(confusion, index=['True', 'False'], columns=['True', 'False'])

    print("\n오차행렬:")
    display(confusion_df)
    print("평가 지표:")
    display(metrics)


In [115]:
pred = model.predict(x_val)
get_clf_eval(y_val, pred)


오차행렬:


,True,False
True,739,208
False,66,10847


평가 지표:


,정확도,정밀도,재현율,F1 Score,Micro F1
0,0.976897,0.918012,0.780359,0.843607,0.976897


## 4. 제출하기

### 테스트 데이터 예측

In [119]:
# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

In [120]:
test_pred = model.predict(x_test)
sum(test_pred) # True로 예측된 개수

509

### 제출 파일 작성

In [121]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission_model.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**

.